In [1]:
import pandas as pd

In [3]:
df = pd.read_csv('combined_scrape.csv')

In [4]:
df.shape

(1808142, 4)

In [5]:
df.head()

,Comment Text,Author,Subreddit,Post ID
0,"Just curious, but what triggered your feelings...",PDXorax,YangForPresidentHQ,ew5ohva
1,Christmas time put up a light-up Yang sign!,PDXorax,YangForPresidentHQ,ew5o8vv
2,"During the general election, get in contact wi...",PDXorax,YangForPresidentHQ,ew42bdx
3,"We need to understand as a country, that the m...",PDXorax,YangForPresidentHQ,evva7l8
4,"Part of the reason for this, is because we nee...",PDXorax,YangForPresidentHQ,evoo179


In [6]:
df.dtypes

Comment Text    object
Author          object
Subreddit       object
Post ID         object
dtype: object

In [7]:
sum_series = df.groupby(['Author','Subreddit']).size()

In [8]:
users = list(set([x for x in sum_series.index.get_level_values(0)]))

In [9]:
len(users)

2882

In [10]:
list_of_subs2 = ['nfl','MMA','CFB','PoliticalHumor','teenagers',
                'soccer','freefolk','gaming','hockey','Libertarian','gameofthrones','MLS',
               ' JoeRogan','conspiracy','marvelstudios','television','asoiaf','leagueoflegends',
                'atheism','samharris','fantasyfootball','CHIBears','hiphopheads',
                'technology','survivor','canada','Futurology','exmormon','Destiny','memes',
                'CryptoCurrency','rupaulsdragrace','Music','dankmemes','stupidpol',
                'Games','FortNiteBR','BigBrother','redsox','baseball','Conservative','Economics',
                'ClashRoyale','DestinyTheGame','europe','NintendoSwitch','IntellectualDarkWeb',
                'askgaybros','hillaryclinton','tuesday','2007scape','tulsi',
                'NASCAR','trees','HistoryMemes','TheSilphRoad','pokemon','Brawlstars','environment','fo76',
                'NYYankees','Android','Overwatch','aznidentity','barstoolsports','wholesomememes',
                'JordanPeterson','TheLastAirbender','golf','DotA2','space','collapse']

In [11]:
features = {}
political_dict={}
for user in users:
    feature_dict = {}
    for sub in sum_series[user].index:
        if sub in list_of_subs2:
            feature_dict[sub]=sum_series[user][sub]
    if len(feature_dict)!=0:
        features[user]=feature_dict
        #create target series
        inner_dict = {}
        try:
            inner_dict['yang'] = sum_series[user]['YangForPresidentHQ']
        except:
            inner_dict['yang'] = 0
        try:
            inner_dict['bernie'] = sum_series[user]['SandersForPresident']
        except:
            inner_dict['bernie'] = 0
        try:
            inner_dict['bernie'] += sum_series[user]['wayofthebern']
        except:
            inner_dict['bernie'] += 0
        try:
            inner_dict['bernie'] += sum_series[user]['Political_Revolution']
        except:
            inner_dict['bernie'] += 0
        try:
            inner_dict['pete'] = sum_series[user]['Pete_Buttigieg']
        except:
            inner_dict['pete'] = 0
        try:
            inner_dict['kamala'] = sum_series[user]['Kamala']
        except:
            inner_dict['kamala'] = 0
        try:
            inner_dict['biden'] = sum_series[user]['JoeBiden']
        except:
            inner_dict['biden'] = 0
        political_dict[user]=inner_dict


In [12]:
df_feature = pd.DataFrame(features).T

In [13]:
df_target = pd.DataFrame(political_dict).T

In [14]:
def func(row):
    total = sum(row)
    if (row['bernie']/total >= .8) & (row['bernie']>=4):
        return 'bernie'
    if (row['biden']/total >= .8) & (row['biden']>=4):
        return 'biden'
    if (row['kamala']/total >= .8) & (row['kamala']>=4):
        return 'kamala'
    if (row['pete']/total >= .8) & (row['pete']>=4):
        return 'pete'
    if (row['yang']/total >= .8) & (row['yang']>=4):
        return 'yang'
    return 'unknown'

In [15]:
df_target['target'] = df_target.apply(func,axis=1)

In [16]:
df_target.drop(['bernie','biden','kamala','pete','yang'],axis=1,inplace=True)

In [18]:
df_target.target.value_counts()

unknown    743
bernie     655
yang       502
pete       312
kamala      73
biden       60
Name: target, dtype: int64

In [19]:
df_feature.head()

,2007scape,Android,BigBrother,Brawlstars,CFB,CHIBears,ClashRoyale,Conservative,CryptoCurrency,Destiny,...,space,stupidpol,survivor,technology,teenagers,television,trees,tuesday,tulsi,wholesomememes
iredditallman,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ux2o2h,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Recon_Figure,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Karl_Marx_,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
hannibe,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,11.0,1.0,NaN,NaN,NaN,6.0


In [20]:
df_feature.fillna(0,inplace=True)

In [21]:
#add code to combine like minded subs
df_feature['got']=df_feature['gameofthrones']+df_feature['asoiaf']+df_feature['freefolk']
df_feature['soccer_comb']=df_feature['MLS']+df_feature['soccer']
df_feature['football']=df_feature['nfl']+df_feature['CFB']+df_feature['CHIBears']
df_feature['baseball_comb']=df_feature['baseball']+df_feature['redsox']+df_feature['NYYankees']
df_feature['conservative_comb']=df_feature['Conservative']+df_feature['tuesday']
df_feature['games_comb']=df_feature['gaming']+df_feature['Games']
df_feature['memes_comb']=df_feature['memes']+df_feature['dankmemes']+df_feature['HistoryMemes']+df_feature['wholesomememes']
df_feature['pokemon_comb']=df_feature['TheSilphRoad']+df_feature['pokemon']

In [22]:
#remove above single subs
df_feature.drop(['redsox','baseball','NYYankees','Conservative','tuesday','nfl','CFB','CHIBears','gaming','Games','freefolk','gameofthrones',
                 'asoiaf','memes','dankmemes','HistoryMemes','wholesomememes','TheSilphRoad','pokemon','soccer',
                 'MLS'],axis=1,inplace=True)

In [23]:
df_feature = df_feature.div(df_feature.sum(axis=1), axis=0)

In [24]:
df_feature.head(20)

,2007scape,Android,BigBrother,Brawlstars,ClashRoyale,CryptoCurrency,Destiny,DestinyTheGame,DotA2,Economics,...,trees,tulsi,got,soccer_comb,football,baseball_comb,conservative_comb,games_comb,memes_comb,pokemon_comb
iredditallman,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,1.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
ux2o2h,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
Recon_Figure,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,1.000000,0.000000
Karl_Marx_,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.240310,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
hannibe,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.272727,0.000000
snylack,0.0,0.0,0.0,0.0,0.0,0.010753,0.0,0.0,0.000000,0.005376,...,0.0,0.000000,0.000000,0.000000,0.005376,0.0,0.016129,0.000000,0.016129,0.000000
webbess1,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
Twinkie60,0.0,0.0,0.0,0.0,0.0,0.777778,0.0,0.0,0.027778,0.000000,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
Ramietoes,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.000000,0.018868,0.056604,0.0,0.000000,0.075472,0.150943,0.000000
mckenny37,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,...,0.0,0.000000,0.216216,0.000000,0.000000,0.0,0.000000,0.018018,0.009009,0.000000


In [25]:
df = pd.concat([df_feature,df_target],axis=1)

In [26]:
df.shape

(2345, 59)

In [27]:
df=df[df['target']!='unknown']

In [28]:
df.shape

(1602, 59)

In [29]:
X = df.drop(['target'],axis=1)

In [30]:
y = df['target']

In [31]:
y.value_counts()

bernie    655
yang      502
pete      312
kamala     73
biden      60
Name: target, dtype: int64

In [ ]:
#removing the biden and kamala members since they're counts are really low

In [32]:
df2 = df[(df['target']!='kamala')&(df['target']!='biden')]

In [33]:
df2.shape

(1469, 59)

In [34]:
from sklearn.model_selection import train_test_split

In [35]:
X2 = df2.drop(['target'],axis=1)

In [36]:
y2 = df2['target']

In [37]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2,stratify = y2, random_state=42)

In [38]:
from sklearn.linear_model import LogisticRegression

In [39]:
lr2 = LogisticRegression(multi_class='multinomial',solver='lbfgs')

In [40]:
lr2.fit(X2_train,y2_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='multinomial', n_jobs=None, penalty='l2',
                   random_state=None, solver='lbfgs', tol=0.0001, verbose=0,
                   warm_start=False)

In [41]:
lr2.score(X2_train,y2_train)

0.555858310626703

In [42]:
lr2.score(X2_test,y2_test)

0.5353260869565217

In [43]:
results = pd.DataFrame(lr2.coef_.T,X2.columns,lr2.classes_)

In [44]:
y2.value_counts(normalize=True)

bernie    0.445882
yang      0.341729
pete      0.212389
Name: target, dtype: float64

In [47]:
results

,bernie,pete,yang
2007scape,-0.110895,-0.309533,0.420428
Android,0.064920,-0.380094,0.315174
BigBrother,0.065116,0.641085,-0.706201
Brawlstars,-0.323310,-0.175595,0.498905
ClashRoyale,-0.376082,0.173716,0.202366
CryptoCurrency,-0.420468,-0.408066,0.828534
Destiny,0.460518,0.122210,-0.582728
DestinyTheGame,0.133868,-0.063350,-0.070518
DotA2,-0.381512,0.168067,0.213445
Economics,-0.132698,0.887516,-0.754819


In [56]:
results[abs(results['yang']>1)]

,bernie,pete,yang
Futurology,-0.180499,-1.201464,1.381962
aznidentity,-1.277873,-0.671660,1.949533
samharris,-0.805993,-0.320526,1.126519
